# preferred data

In [1]:
import csv
import re

In [2]:
# skills 데이터 다 확인

def getSkillsSet():
    with open("../data/crawled_data_combined.csv", encoding="utf-8") as f:
        reader = csv.reader(f)

        # 태그 하나씩 쪼개서 넣을 리스트 (set으로 중복 제거 전)
        skillList = []

        # csv 파일 리더로, 한 row씩 읽기
        for row in reader:

            # row list의 10번째 element인 tags string을 ''안 단어들로 나눠서 list만들기
            skills = re.findall(r"'(.*?)'", row[10])     
            skills_upper = [word.upper() for word in skills]

            # 각 줄의 태그를 tagList에 넣는다.
            skillList += skills_upper
            # print(skills)

        skillsSet = set(skillList)
        
        return skillsSet

####  ListToDict(lst) - skills List로 딕셔너리 만들기

In [3]:
def ListToDict(lst):
    toDict = dict()

    for item in lst:
        toDict[item.upper()] = 0
    
    return toDict
    


#### saveCSV(dict, title) - dictionary key, value를 csv 파일로 저장하는 함수

In [4]:
import csv

def saveCSV(dict, title):
    # Specify the CSV file path

    # Open the CSV file in write mode
    with open(title, "w", newline="") as file:
        writer = csv.writer(file)

        # Write the header row with column names
        writer.writerow(["Key", "Value"])

        # Write each key-value pair as a row in the CSV file
        for key, value in dict.items():
            writer.writerow([key, value])

    print("Dictionary keys and values saved to CSV file successfully.")

In [5]:
comSet = getSkillsSet()
comList = list(comSet)
comDict = ListToDict(comList)

### 각 채용링크의 qualification 속 skill set 정리하기

In [6]:
import re
from tqdm import tqdm
import csv
from konlpy.tag import Komoran

def preferred():
    skillDict = comDict.copy()
    freq_dict = dict()

    with open("../data/crawled_data_combined.csv", 'r', encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        for row in tqdm(data[1:]):
            id = row[0]
            data = row[8].upper()

            # 확인해야 하는 데이터 (list)
            phrases = comList 

            # remove symbols
            sentence = re.sub(r'[^\w\s]', '', data)

            # re.escape으로 특수문자 벗어나고, |로 단어 연결
            pattern = r'\b(' + '|'.join(map(re.escape, phrases)) + r')\b'

            # 한글 정렬후 나누기
            komoran = Komoran()
            sentence = komoran.morphs(sentence)

            # 한글 정렬 후 나누고 다시 문장으로 합쳐주기
            sentence = " ".join(sentence)
            
            # phrases에 있는 단어 sentence속 빈도 확인하기
            matches = re.findall(pattern, sentence)
            

            if count % 500 == 0:
                print(matches)
            count += 1

            # dictionary에 하나씩 넣어주기 - set으로 중복 제거
            freq_dict[id] = list(set(matches))

    saveCSV(freq_dict, "freq_pref_20230703.csv")
    return freq_dict
            

# qualifications = 7, preferred = 8, skills = 10

In [7]:
preferred()

  0%|          | 10/5451 [00:35<5:25:30,  3.59s/it]


KeyboardInterrupt: 